Luego de crear todos los datasets con la información más importante, crearemos uno que junte toda la información en un mismo lugar.

In [9]:
import pandas as pd
import json
import parquet
import numpy as np
from sklearn.impute import KNNImputer

%load_ext autoreload
%autoreload 2

import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Vamos a cargar todos los archivos primero.

In [10]:
# Lista con las rutas de los archivos
file_paths = [
    '../Datasets_limpios/siniestros2018.csv',
    '../Datasets_limpios/siniestros2019.csv',
    '../Datasets_limpios/siniestros2020.csv',
    '../Datasets_limpios/siniestros2021.csv',
    '../Datasets_limpios/siniestros2022.csv',
    '../Datasets_limpios/siniestros2023.csv',
    '../Datasets_limpios/siniestros2024.csv',    
]

# Leer todos los archivos CSV y almacenarlos en una lista de dataframes
dataframes = [pd.read_csv(file_path) for file_path in file_paths]

# Concatenar todos los dataframes en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Mostrar las primeras filas del dataset combinado
combined_df.head()

,Unnamed: 0,id_feu,siniestro_fecha,siniestro_hora,zona_ocurrencia,via_publica,nombre_via,altura_km,entre_calle_1,latitud,...,estado_fisico_ambiental,luminosidad,luz_artificial,estado_ambiental,visibilidad,semaforo,tipo_involucrado,tipo_colision,cono_de_vision,barreras_de_seguridad
0,0,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Motocicleta,NaN,NaN,NaN
1,1,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Motocicleta,NaN,NaN,NaN
2,2,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Automóvil,NaN,NaN,NaN
3,3,49059,2018-10-31,19:30:00,Urbana,Calle,BELASCOAIN,900,VIUDES,-2.748.406,...,"Despejada, Seca",Noche,NO,Nublado,Buena,Sin Semáforo,Motocicleta,NaN,NaN,NaN
4,4,49059,2018-10-31,19:30:00,Urbana,Calle,BELASCOAIN,900,VIUDES,-2.748.406,...,"Despejada, Seca",Noche,NO,Nublado,Buena,Sin Semáforo,Automóvil,NaN,NaN,NaN


Verificamos los datos una vez más

In [11]:
utils.verificar_tipo_datos(combined_df)
combined_df

,Unnamed: 0,id_feu,siniestro_fecha,siniestro_hora,zona_ocurrencia,via_publica,nombre_via,altura_km,entre_calle_1,latitud,...,estado_fisico_ambiental,luminosidad,luz_artificial,estado_ambiental,visibilidad,semaforo,tipo_involucrado,tipo_colision,cono_de_vision,barreras_de_seguridad
0,0,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Motocicleta,NaN,NaN,NaN
1,1,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Motocicleta,NaN,NaN,NaN
2,2,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,4200,LAS PIEDRAS,-2.748.105,...,Seca,Atardecer,NaN,Despejado,Buena,Funciona,Automóvil,NaN,NaN,NaN
3,3,49059,2018-10-31,19:30:00,Urbana,Calle,BELASCOAIN,900,VIUDES,-2.748.406,...,"Despejada, Seca",Noche,NO,Nublado,Buena,Sin Semáforo,Motocicleta,NaN,NaN,NaN
4,4,49059,2018-10-31,19:30:00,Urbana,Calle,BELASCOAIN,900,VIUDES,-2.748.406,...,"Despejada, Seca",Noche,NO,Nublado,Buena,Sin Semáforo,Automóvil,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6875,271,220443,2024-01-06,6:00:00,Urbana,Avenida,AV MAIPU,3100,TILCARA,NaN,...,Seca,Noche,NO,Despejado,NaN,No Funciona,Motocicleta,NaN,S/D,Ninguna
6876,272,229168,2024-01-04,8:30:00,Urbana,Avenida,AV 3 DE ABRIL,1300,CORDOBA,NaN,...,Mojada,Día,NaN,Lluvia,NaN,Funciona,Automóvil,NaN,S/D,Ninguna
6877,273,229168,2024-01-04,8:30:00,Urbana,Avenida,AV 3 DE ABRIL,1300,CORDOBA,NaN,...,Mojada,Día,NaN,Lluvia,NaN,Funciona,Motocicleta,NaN,S/D,Ninguna
6878,274,217505,2024-01-01,12:15:00,Urbana,Ruta Nacional,12,1000,COLECTORA (BOCA UNIDOS),NaN,...,Seca,Día,NaN,Despejado,NaN,Sin Semáforo,Automóvil,NaN,S/D,Asfalto


Eliminamos lo más nulo y la columna Unnamed

In [12]:
# Eliminar columnas con más del 80% de valores nulos
umbral = 0.8  # Umbral del 80%
combined_df = combined_df.dropna(thresh=len(combined_df) * (1 - umbral), axis=1)

combined_df = combined_df.drop(columns=['Unnamed: 0'])

# Mostrar las columnas que se mantuvieron
combined_df.columns

Index(['id_feu', 'siniestro_fecha', 'siniestro_hora', 'zona_ocurrencia',
       'via_publica', 'nombre_via', 'altura_km', 'entre_calle_1', 'latitud',
       'longitud', 'ilesos', 'heridos', 'fallecidos', 'vehiculos', 'peatones',
       'tipo_siniestro_unico', 'cantidad_de_involucrados',
       'material_de_la_calzada', 'estado_de_la_calzada',
       'estado_fisico_ambiental', 'luminosidad', 'luz_artificial',
       'estado_ambiental', 'visibilidad', 'semaforo', 'tipo_involucrado'],
      dtype='object')

Todavía quedan errores por solucionar en nuestro dataset. Primero, solucionaremos las columnas de fecha y hora para poder darles un mejor manejo.

In [13]:
# Primero, aseguramos que la columna está en formato de fecha
combined_df['siniestro_fecha'] = pd.to_datetime(combined_df['siniestro_fecha'])

# Creamos las nuevas columnas
combined_df['anio'] = combined_df['siniestro_fecha'].dt.year
combined_df['mes'] = combined_df['siniestro_fecha'].dt.month
combined_df['dia'] = combined_df['siniestro_fecha'].dt.day

# Si la columna 'siniestro_hora' es un string, convertimos a formato datetime
combined_df['siniestro_hora'] = pd.to_datetime(combined_df['siniestro_hora'], format='%H:%M:%S').dt.time

# Visualizar el resultado
print(combined_df.head())

   id_feu siniestro_fecha siniestro_hora zona_ocurrencia via_publica  \
0   49056      2018-10-29       20:10:00          Urbana     Avenida   
1   49056      2018-10-29       20:10:00          Urbana     Avenida   
2   49056      2018-10-29       20:10:00          Urbana     Avenida   
3   49059      2018-10-31       19:30:00          Urbana       Calle   
4   49059      2018-10-31       19:30:00          Urbana       Calle   

         nombre_via altura_km entre_calle_1     latitud    longitud  ...  \
0  AV INDEPENDENCIA      4200   LAS PIEDRAS  -2.748.105  -5.880.054  ...   
1  AV INDEPENDENCIA      4200   LAS PIEDRAS  -2.748.105  -5.880.054  ...   
2  AV INDEPENDENCIA      4200   LAS PIEDRAS  -2.748.105  -5.880.054  ...   
3        BELASCOAIN       900        VIUDES  -2.748.406  -5.877.596  ...   
4        BELASCOAIN       900        VIUDES  -2.748.406  -5.877.596  ...   

   estado_fisico_ambiental  luminosidad  luz_artificial  estado_ambiental  \
0                     Seca    Ata

Por último, solucionamos con código los valores faltantes en latitud y longitud para poder graficar.

In [14]:
# Eliminar puntos innecesarios y convertir a float
combined_df['latitud'] = combined_df['latitud'].str.replace('.', '', regex=False).astype(float)
combined_df['longitud'] = combined_df['longitud'].str.replace('.', '', regex=False).astype(float)

# Dividir por 100000 para obtener las coordenadas en el formato correcto
combined_df['latitud'] = combined_df['latitud'] / 100000
combined_df['longitud'] = combined_df['longitud'] / 100000

coords_df = combined_df[['latitud', 'longitud']]
# Usar KNN para imputar valores nulos
imputer = KNNImputer(n_neighbors=3)
coords_imputed = imputer.fit_transform(coords_df)

# Redondear los valores imputados a 5 decimales
coords_imputed = np.round(coords_imputed, 7)

# Actualizar el dataframe con los valores imputados y redondeados
combined_df[['latitud', 'longitud']] = coords_imputed

# Verificar los primeros resultados
print(combined_df[['latitud', 'longitud']].head())


    latitud  longitud
0 -27.48105 -58.80054
1 -27.48105 -58.80054
2 -27.48105 -58.80054
3 -27.48406 -58.77596
4 -27.48406 -58.77596


In [15]:
# Guardar el dataset combinado en un nuevo archivo Parquet, para que mantenga sus types.
combined_df.to_parquet('../Datasets_limpios/siniestrostotales.parquet', index=False)